In [ ]:
import multiprocessing as mp
import matplotlib.pyplot as plt
import deepdish as dd
from brian2.only import *
import brian2genn

%load_ext autoreload
%autoreload 1

%aimport numpy_
%aimport spatial
%aimport model
%aimport inputs
%aimport util
%aimport spike_utils
%aimport readout

# for the IDE:
import numpy_ as np
import spatial, model, inputs, readout

from util import concatenate
np.concatenate = concatenate
from spike_utils import iterspikes

In [ ]:
rng = np.random.default_rng()
set_device('genn')

In [ ]:
N = 1000
inhibitory_ratio = .2

params = {
    # Simulation
    'dt': 1*ms,
    
    # Population size
    'N': N,
    'N_exc': int((1-inhibitory_ratio) * N),
    'N_inh': int(inhibitory_ratio * N),

    # Membrane
    'v_rest': -60*mV,
    'v_reset': -74*mV,
    'v_threshold': -54*mV,
    'voltage_init': 'v_rest',
    
    'tau_mem': 30*ms,
    'refractory_exc': 3*ms,
    'refractory_inh': 2*ms,

    # Threshold adaptation - Exc
    'th_tau': 1*second,
    'th_ampl': 1*mV,

    # Short-term plasticity - Exc
    'tau_rec': 150*msecond,
    'U': 0.4,

    # Synapse dynamics
    'E_exc': 0*mV,
    'tau_ampa': 2*msecond,
    'E_inh': -100*mV,
    'tau_gaba': 4*msecond,
    
    # # Stochasticity
    # 'tau_noise': 10*msecond,
    # 'vnoise_std': 0.5*mV,

    # Layout
    'r_dish': 4*mm,
    'weight_distribution': 'singular',
    
    # Connectivity: Inh
    'r_inh': 1*mm,
    'outdeg_inh': 50,
    'w_inh_mean': 1,

    # Connectivity: Exc
    'r_exc': 2*mm,
    'outdeg_exc': 50,
    'w_exc_mean': 1,

    # Stimulus
    'N_stimuli': 5,
    'stim_distribution_radius': 2.5*mm,
    'neurons_per_stim': 10,
    'input_strength': 100,

    # Paradigm
    'settling_period': 1*second,
    'sequence_length': 5,
    'sequence_count': 20,
    'fully_random_msc': True,
    'ISI': 100*ms,
}

In [ ]:
fname = 'presynaptic_events_singular.npz'
out_fname = 'data/sweep_isi_0.{isi}.h5'

In [ ]:
f = np.load(fname)
W, X, Y, D, A, B = [f[k] for k in 'WXYDAB']
X, Y = X*meter, Y*meter
Xstim, Ystim = spatial.create_stimulus_locations(params)

In [ ]:
iterations = 10
for isi in np.arange(50, 750, 50):
    isi_params = {**params, 'ISI': isi*ms}
    results = [None]*iterations
    for iteration in range(iterations):
        device.reinit()
        device.activate()
        Net = model.create_network(X, Y, Xstim, Ystim, W, D, isi_params, reset_dt=inputs.get_episode_duration(isi_params))
        results[iteration], T = readout.setup_run(Net, isi_params, rng, {'A': A, 'B': B})
        Net.run(T)
        readout.get_results(Net, isi_params, W, results[iteration])
    dd.io.save(out_fname.format(isi=isi), results)